In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Rectangle
from matplotlib.cm import inferno, rainbow,gist_rainbow_r,jet, ScalarMappable
from matplotlib.colors import Normalize, LinearSegmentedColormap
from scipy.ndimage import median_filter
from skimage.measure import block_reduce
from mpl_toolkits.axes_grid1 import make_axes_locatable


excel_file = "l_serine_behaviour.xlsx"
radius = 150
part = [0,60] ## 0,60 for CX 30,150 for l serine
main_folder = "Insert your parent folder path here"
plot = True





def mintoframes(minutes):
    return minutes*25*60
def check_seq_len(df,corners):
    current_value = 0
    for corner in corners:
        next_value = df[corner][0]
        if current_value > next_value:
            raise ValueError("Incorrect corner positions")
        current_value = next_value

excel_df = pd.DataFrame(columns = ["Date","Day","Cage","Treatment","Distance traveled","Time in center","Angles"])

folders = [os.path.join(main_folder,folder) for folder in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder,folder))]

for folder in folders:
    files = [file for file in os.listdir(folder) if file.endswith("filtered.h5")]
    for file in files:
        print(folder + "\\" + file)
        df = pd.read_hdf(folder + "\\" + file)
        df = df.droplevel(0,axis = 1)
        test = df.loc[:,(slice(None),slice(None),"y")]
        test = 1024-test
        df.loc[:,(slice(None),slice(None),"y")] = test
        corner_x= df.xs(("single","x"),level = [0,2],axis = 1)
        corner_y= df.xs(("single","y"),level = [0,2],axis = 1)
        corner_x = pd.DataFrame(np.nanmedian(corner_x.to_numpy(),axis = 0).reshape((1,16)),columns = corner_x.columns)
        corner_y = pd.DataFrame(np.nanmedian(corner_y.to_numpy(),axis = 0).reshape((1,16)),columns = corner_y.columns)

        corner_y["c3_bottom_left"].loc[0] = corner_y["c3_bottom_right"].loc[0]
        corner_distances = np.array([np.abs(corner_x["c2_top_left"][0] - corner_x["c1_top_right"][0]),np.abs(corner_x["c3_top_left"][0] - corner_x["c4_top_right"][0])])
        if np.abs(corner_distances[0]-corner_distances[1]) < 50:
            corner_distance = np.mean(corner_distances)/0.8
        else:
            raise ValueError
        
        check_seq_len(corner_x,["c3_top_left","c3_top_right","c4_top_left","c4_top_right"])
        check_seq_len(corner_x,["c3_bottom_left","c3_bottom_right","c4_bottom_left","c4_bottom_right"])
        check_seq_len(corner_x,["c2_top_left","c2_top_right","c1_top_left","c1_top_right"])
        check_seq_len(corner_x,["c3_bottom_left","c3_bottom_right","c4_bottom_left","c4_bottom_right"])

        check_seq_len(corner_y,["c2_bottom_left","c2_top_left","c3_bottom_left","c3_top_left"])
        check_seq_len(corner_y,["c2_bottom_right","c2_top_right","c3_bottom_right","c3_top_right"])

        check_seq_len(corner_y,["c1_bottom_left","c1_top_left","c4_bottom_left","c4_top_left"])
        check_seq_len(corner_y,["c1_bottom_right","c1_top_right","c4_bottom_right","c4_top_right"])




        mouse_numbers = [ind for ind in df.columns.get_level_values(0).unique()if ind.startswith("m")]

        init_xs = []
        init_ys = []
        m_dict = {
            "c1":-1,
            "c2":-1,
            "c3":-1,
            "c4":-1,
        }
        color_dict = {
            "c1":"black",
            "c2":"blue",
            "c3":"green",
            "c4":"pink",
        }
        for mouse in mouse_numbers:
            m_df_x = median_filter(df.xs((mouse,"x"),level = [0,2] ,axis = 1).to_numpy()[mintoframes(part[0]):mintoframes(part[1]),:],size = 1,axes = 0)
            m_df_y = median_filter(df.xs((mouse,"y"),level = [0,2] ,axis = 1).to_numpy()[mintoframes(part[0]):mintoframes(part[1]),:],size = 1,axes = 0)
            init_x = np.nanmean(m_df_x)
            init_y = np.nanmean(m_df_y)
            cage = np.sqrt((corner_x-init_x)**2+(corner_y-init_y)**2).loc[0].idxmin()[:2]
            m_dict[cage] = mouse
            init_xs.append(init_x)
            init_ys.append(init_y)
        center_square_df = np.empty((4,len(m_df_x),8))
        distance_moved = np.empty((4,len(m_df_x)-1,8))
        angles = np.empty((4,len(m_df_x)))
        for idx,cage in enumerate(m_dict):
            if m_dict[cage] != -1:
                m_df_x = median_filter(df.xs((m_dict[cage],"x"),level = [0,2] ,axis = 1).to_numpy()[mintoframes(part[0]):mintoframes(part[1]),:],size = 1,axes = 0)
                m_df_y = median_filter(df.xs((m_dict[cage],"y"),level = [0,2] ,axis = 1).to_numpy()[mintoframes(part[0]):mintoframes(part[1]),:],size = 1,axes = 0)

                m_arr = np.stack((m_df_x,m_df_y))
                distance_moved[idx,:,:] = np.linalg.norm(np.diff(m_arr/corner_distance,axis = 1),axis = 0)
                print(distance_moved[idx,:,0].shape)

                corner_lst = []

                corner_lst.append((corner_x[f"{cage}_top_left"][0],corner_y[f"{cage}_top_left"][0]))
                corner_lst.append((corner_x[f"{cage}_top_right"][0],corner_y[f"{cage}_top_right"][0]))
                corner_lst.append((corner_x[f"{cage}_bottom_right"][0],corner_y[f"{cage}_bottom_right"][0]))
                corner_lst.append((corner_x[f"{cage}_bottom_left"][0],corner_y[f"{cage}_bottom_left"][0]))
                center = np.mean(np.array(corner_lst),axis = 0)
                distance_norm = Normalize(vmin = 0,vmax = 10)
                # cmap_colors = ["blue","red"]
                # cmap = LinearSegmentedColormap.from_list("custom", cmap_colors)
                cmap = jet
                distance_colors = cmap(distance_norm(distance_moved[idx,:,0]*100*25))
                if plot:
                    plt.scatter(m_df_x[:-1,0],m_df_y[:-1,0],s = 4,alpha = .1,linewidth = 0,c = distance_colors)
                    corner_radii = np.mean(np.abs(np.array(corner_lst)-center),axis = 0)
                    ax = plt.gca()
                    ax.add_patch(Rectangle(center-corner_radii,corner_radii[0]*2,corner_radii[1]*2,edgecolor = "black",linewidth = 1,fill = False,zorder = -10,linestyle="dashed"))
                    ax.add_patch(Rectangle(center-radius,radius*2,radius*2,edgecolor = "black",linewidth = 1,fill = False,linestyle="dashed"))
                    ax.patch.set_visible(False)
                    ax.axis('off')
                center_square_df[idx,:,:] = np.max(np.abs(m_arr-center.reshape((2,1,1))),axis = 0)
                v_center = center.reshape((2,1)) - m_arr[:,:,1]
                v_head = m_arr[:,:,0]-m_arr[:,:,1]
                dp = np.einsum('ij,ij->j',v_head,v_center)
                angles[idx,:] = np.degrees(np.arccos(dp/(np.linalg.norm(v_head,axis = 0)*np.linalg.norm(v_center,axis = 0))))
                if plot:
                    print("Cage: ",cage)
                    print("Moved: ",np.nansum(distance_moved[idx,:,0]))
                    print("Time in center: ", np.mean(center_square_df[idx,:,:]<radius))
                bins = np.linspace(0,part[1]-part[0],part[1]-part[0]//10+1)
                treatment = "?"
                day = "?"
                if "Day1" in file and (cage == "c1" or cage == "c4"):
                    day = "Day1"
                    treatment = "Saline"    
                if "Day1" in file and (cage == "c2" or cage == "c3"):
                    day = "Day1"
                    treatment = "CX"    
                if "Day2" in file and (cage == "c1" or cage == "c4"):
                    day = "Day2"
                    treatment = "CX"    
                if "Day2" in file and (cage == "c2" or cage == "c3"):
                    day = "Day2"
                    treatment = "Saline"    
                if "Hab" in file and (cage == "c1" or cage == "c4"):
                    day = "Hab"
                    treatment = "Hab"    
                if "Hab" in file and (cage == "c2" or cage == "c3"):
                    day = "Hab"
                    treatment = "Hab"    
                excel_df.loc[len(excel_df)] = [os.path.basename(folder),os.path.basename(file).split(".")[0].split("_")[1],cage,treatment,
                block_reduce(distance_moved[idx,:,0], block_size=(25*60*10), func=np.nansum, cval=np.nanmean(distance_moved[idx,-1000:,0])),
                block_reduce(center_square_df[idx,:,:]<radius,block_size=(25*60*10,8), func=np.nanmean, cval=np.nanmean(center_square_df[idx,:,:]<radius)).flatten(),
                ','.join(map(str,np.histogram(angles[idx,:],bins)[0]))]
        plt.xlim([0,1280])
        plt.ylim([0,1024])
        xy_scale = 1024/1280
        plt.xlim([-20,-20+650])
        plt.ylim([0,650*xy_scale])
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(ScalarMappable(norm = distance_norm,cmap = cmap),cax = cax)
        plt.savefig(f"{os.path.basename(folder)}_{file} tracking.png",dpi = 300)
        plt.show()
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
excel_df.sort_values(by=['Date','Day', 'Treatment'],inplace = True)
excel_df.to_excel(writer, sheet_name='Sheet1')
workbook = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.set_column('A:A', 30)
worksheet.set_column('B:F', 15)
worksheet.set_column('G:H', 25)

writer.close()
del writer
        
def barplot_annotate_brackets(num1, num2, data, center, height, yerr=None, dh=.05, barh=.1, fs=None, maxasterix=None):


    if type(data) is str:
        text = data
    else:
        # * is p < 0.05
        # ** is p < 0.005
        # *** is p < 0.0005
        # etc.
        text = ''
        p = .05

        while data < p:
            text += '*'
            p /= 10.

            if maxasterix and len(text) == maxasterix:
                break

        if len(text) == 0:
            text = 'n. s.'

    lx, ly = center[num1], height[num1]
    rx, ry = center[num2], height[num2]

    if yerr:
        ly += yerr[num1]
        ry += yerr[num2]

    ax_y0, ax_y1 = plt.gca().get_ylim()
    dh *= (ax_y1 - ax_y0)
    barh *= (ax_y1 - ax_y0)

    y = max(ly, ry) + dh

    barx = [lx, lx, rx, rx]
    bary = [y, y+barh, y+barh, y]
    mid = ((lx+rx)/2, y+barh)

    plt.plot(barx, bary, c='black')

    kwargs = dict(ha='center', va='bottom')
    if fs is not None:
        kwargs['fontsize'] = fs

    plt.text(*mid, text, **kwargs)



L-serine figures

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu, sem, wilcoxon
import random
random.seed(1234)
fig,ax = plt.subplots(2,1)
all_data = np.zeros((2,2,8,13))
ylabels = ["Time spent in center (%)","Distance moved (m)"]
colors = ["#D70089","gray"]
for treat_idx,(treatment,group) in enumerate(excel_df.groupby("Treatment")):
    for idx,quant in enumerate(["Time in center","Distance traveled"]):
        data = np.vstack(group[quant])
        if quant == "Time in center":
            all_data[0,treat_idx,:,:] = data
        if quant == "Distance traveled":
            all_data[1,treat_idx,:,:] = data
        current_ax = ax[idx]
        if quant == "Time in center":
            current_ax.errorbar(list(range(30,160,10)),np.nanmean(data,axis = 0)*100,yerr = sem(data*100,axis = 0),label =treatment,color = colors[treat_idx])
        else:
            current_ax.errorbar(list(range(30,160,10)),np.nanmean(data,axis = 0),yerr = sem(data,axis = 0),label =treatment,color = colors[treat_idx])
            plt.ylim([0,5.5])
            current_ax.text(100,1.85,"*",horizontalalignment="center")
        current_ax.set_xlabel("Time (min)")
        current_ax.set_ylabel(ylabels[idx])
        current_ax.set_xticks(list(range(30,160,10)))
fig.tight_layout()
plt.savefig("l_serine Behavior.svg",dpi = 300)
plt.show()

fig,ax = plt.subplots(2,1,figsize = (3,9))
treat_idx = 0
for treatment,group in excel_df.groupby("Treatment"):
    for idx,quant in enumerate(["Time in center","Distance traveled"]):
        data = np.vstack(group[quant])
        print(f"{np.mean(data)}+-{sem(data,axis = None)}")
        current_ax = ax[idx]
        if quant == "Time in center":
            current_ax.bar(treat_idx,np.mean(data)*100,yerr = sem(data,axis = None)*100,ecolor = colors[treat_idx],edgecolor = colors[treat_idx], fill = False,capsize = 5)
            current_ax.scatter([treat_idx+random.uniform(-0.1,0.1) for x in range(data.shape[0])], np.mean(data,axis = 1)*100,color = colors[treat_idx])
        else:
            plt.ylim([0,5])
            current_ax.bar(treat_idx,np.mean(data),yerr = sem(data,axis = None),ecolor = colors[treat_idx],edgecolor = colors[treat_idx], fill = False,capsize = 5)
            current_ax.scatter([treat_idx+random.uniform(-0.1,0.1) for x in range(data.shape[0])], np.mean(data,axis = 1),color = colors[treat_idx])
            # if treat_idx == 0:
            #     barplot_annotate_brackets(0,1,0.03,[0,1],[2,1],dh = 0.31)
        current_ax.set_xticks([0,1])
        current_ax.set_xticklabels(["L-Serine","Saline"])
        current_ax.set_ylabel(ylabels[idx])
    treat_idx += 1


fig.tight_layout()
plt.savefig("l_serine bar behaviour.svg",dpi = 300)
plt.show()
quant_idx = 1
# wilcoxon(all_data[quant_idx,0,:,:],all_data[quant_idx,1,:,:],axis = 0)
wilcoxon(np.mean(all_data[quant_idx,0,:,:],axis = 1),np.mean(all_data[quant_idx,1,:,:],axis = 1),axis = 0)



CX figures

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu, sem, wilcoxon
import random
random.seed(1234)
fig,ax = plt.subplots(2,1)
all_data = np.zeros((2,2,10,6))
ylabels = ["Time spent in center (%)","Distance moved (m)"]
colors = ["#00c8c8","gray"]
excel_df = excel_df[excel_df["Treatment"] != "Hab"]
for treat_idx, (treatment,group) in enumerate(excel_df.groupby("Treatment")):
    for idx,quant in enumerate(["Time in center","Distance traveled"]):
        data = np.vstack(group[quant])
        if quant == "Time in center":
            all_data[0,treat_idx,:,:] = data
        if quant == "Distance traveled":
            all_data[1,treat_idx,:,:] = data
        current_ax = ax[idx]
        if quant == "Time in center":
            current_ax.errorbar(list(range(10,70,10)),np.nanmean(data,axis = 0)*100,yerr = sem(data*100,axis = 0),label =treatment,color = colors[treat_idx])
        else:
            plt.ylim([0,20])
            current_ax.errorbar(list(range(10,70,10)),np.nanmean(data,axis = 0),yerr = sem(data,axis = 0),label =treatment,color = colors[treat_idx])
            # plt.ylim([0,5.5])
            current_ax.text(10,11.5,"*",horizontalalignment="center")
            current_ax.text(50,6,"*",horizontalalignment="center")
        # current_ax.errorbar(list(range(10,70,10)),np.nanmean(data,axis = 0),yerr = sem(data,axis = 0),label =treatment)
        current_ax.set_xlabel("Time (min)")
        current_ax.set_ylabel(ylabels[idx])
        current_ax.set_xticks(list(range(10,70,10)))
    

fig.tight_layout()
plt.savefig("CX Behavior.svg",dpi = 300)
plt.show()

fig,ax = plt.subplots(2,1,figsize = (3,9))

for treat_idx, (treatment,group) in enumerate(excel_df.groupby("Treatment")):

    for idx,quant in enumerate(["Time in center","Distance traveled"]):
        data = np.vstack(group[quant])
        current_ax = ax[idx]
        if quant == "Time in center":
            current_ax.bar(treat_idx,np.mean(data)*100,yerr = sem(data,axis = None)*100,ecolor = colors[treat_idx],edgecolor = colors[treat_idx], fill = False,capsize = 5)
            current_ax.scatter([treat_idx+random.uniform(-0.1,0.1) for x in range(data.shape[0])], np.mean(data,axis = 1)*100,color = colors[treat_idx])
        else:
            current_ax.bar(treat_idx,np.mean(data),yerr = sem(data,axis = None),ecolor = colors[treat_idx],edgecolor = colors[treat_idx], fill = False,capsize = 5)
            current_ax.scatter([treat_idx+random.uniform(-0.1,0.1) for x in range(data.shape[0])], np.mean(data,axis = 1),color = colors[treat_idx])
        current_ax.set_xticks([0,1])
        current_ax.set_xticklabels(["CX 1739","Saline"])
        current_ax.set_ylabel(ylabels[idx])
    treat_idx += 1


fig.tight_layout()
plt.savefig("CX bar behaviour.svg",dpi = 300)
plt.show()


CX figures
